# Welcome to conu demo! 

You need to install following packages to be able to run docker examples: `docker` and `conu`, for kubernetes and OpenShift examples I recommend to run local cluster using `minishift`.

In [ ]:
from IPython.display import display, HTML

# Docker example

In [ ]:
from conu import DockerBackend

In [ ]:
backend = DockerBackend()

In [ ]:
image = backend.ImageClass("docker.io/library/nginx")

In [ ]:
image.inspect()

In [ ]:
container = image.run_via_binary()

In [ ]:
container.is_running()

In [ ]:
container.get_IPv4s()

In [ ]:
container.get_ports()

In [ ]:
container.stop()

In [ ]:
container.is_running()

### HTTP client

In [ ]:
image = backend.ImageClass("registry.fedoraproject.org/fedora")

In [ ]:
container = image.run_via_binary(command=["python3", "-m", "http.server", "--bind", "0.0.0.0 8000"], additional_opts= ["-p", "8000:8000"])

In [ ]:
container.get_ports()

In [ ]:
container.get_IPv4s()

In [ ]:
container.get_port_mappings()

In [ ]:
request = container.http_request(host="127.0.0.1", port="8000")

In [ ]:
request.ok

In [ ]:
display(HTML(request.content.decode("utf-8")))

In [ ]:
request = container.http_request(path="/etc", host="127.0.0.1", port="8000")

In [ ]:
request.ok

In [ ]:
display(HTML(request.content.decode("utf-8")))

In [ ]:
"passwd" in request.content.decode("utf-8")

# Kubernetes example

This example demonstrating how to run image created by `DockerBackend` as container inside Kubernetes pod.

In [ ]:
from conu import K8sBackend

In [ ]:
k8s_backend = K8sBackend()

In [ ]:
docker_backend = DockerBackend()

In [ ]:
image = docker_backend.ImageClass("openshift/hello-openshift")

In [ ]:
pod = image.run_in_pod(namespace='myproject')

In [ ]:
pod.is_ready()

In [ ]:
pod.get_phase()

In [ ]:
pod.delete()

In [ ]:
pod.get_phase()

## Deployment

In [ ]:
from conu.utils import get_oc_api_token

In [ ]:
from conu.backend.k8s.deployment import Deployment

In [ ]:
api_key = get_oc_api_token()
k8s_backend = K8sBackend(api_key=api_key)

In [ ]:
template = """
        apiVersion: apps/v1
        kind: Deployment
        metadata:
          name: hello-world
          labels:
            app: hello-world
        spec:
          replicas: 3
          selector:
            matchLabels:
              app: hello-world
          template:
            metadata:
              labels:
                app: hello-world
            spec:
              containers:
              - name: hello-openshift
                image: openshift/hello-openshift
        """

In [ ]:
test_deployment = Deployment(namespace='myproject', from_template=template, create_in_cluster=True)

In [ ]:
test_deployment.all_pods_ready()

In [ ]:
test_deployment.get_status()

In [ ]:
test_deployment.delete()

# OpenShift example

In [ ]:
from conu.utils import get_oc_api_token

In [ ]:
from conu import OpenshiftBackend

In [ ]:
api_key = get_oc_api_token()

In [ ]:
api_key

In [ ]:
openshift_backend = OpenshiftBackend(api_key=api_key)

In [ ]:
app_name = openshift_backend.deploy_image("centos/mariadb-102-centos7",
                                         oc_new_app_args=["--env", "MYSQL_ROOT_PASSWORD=test"],
                                         project='myproject')

In [ ]:
openshift_backend.all_pods_are_ready(app_name)

In [ ]:
openshift_backend.get_logs(app_name)

In [ ]:
openshift_backend.clean_project(app_name)

## Creating application from source

In [ ]:
openshift_backend.get_status()

In [ ]:
app_name = openshift_backend.create_new_app_from_source("centos/python-36-centos7",
                                                       source="https://github.com/openshift/django-ex.git",
                                                       project="myproject")

In [ ]:
openshift_backend.all_pods_are_ready(app_name)

In [ ]:
openshift_backend.get_status()

In [ ]:
openshift_backend.clean_project(app_name)

For more information see upstream GitHub repository: https://github.com/user-cont/conu. You can find there more examples for all supported backends.